### The **Lorenz system** is a system of ordinary differential equations first studied by mathematician and meteorologist Edward Lorenz. The notable thing about this system is that for a certain subset of parameters and initial conditions it yields chaotic solutions.

### It consist on 3 coupled differential equations.

#### dx/dt = sigma * (y-x)

#### dy/dt = x * (rho-z) - y

#### dz/dt = xy - beta*z

### The idea would be to first solve the Lorenz system numerically. 

In [20]:
# import necessary libraries
import numpy as np
from scipy.integrate import odeint

In [21]:
# Define the Lorenz system
def lorenz(w, t, sigma, rho, beta):
    x, y, z = w
    dwdt = [sigma * (y - x), x * (rho - z) - y, x * y - beta * z]
    return dwdt

In [22]:
# Initial conditions: [x(t = 0), y(t = 0), z(t = 0)]
w0 = [1.0, 0.0, 0.0]

# Define a time vector:
t = np.linspace(0, 30, 3001)

# Define the system parameters
sigma = 10
rho = 28
beta = 2.2

In [23]:
# Solve the system
sol = odeint(lorenz, w0, t, args=(sigma, rho, beta))

## Time Series Plots

### Now that we have the solution for the spatial coordinates, we can see how they change with time.

In [ ]:
import matplotlib.pyplot as plt

x, y, z = sol[:, 0], sol[:, 1], sol[:, 2]

plt.figure(figsize=(12, 8))
plt.subplot(3, 1, 1)
plt.plot(t, x, label='x(t)')
plt.title('Time Series of x')
plt.xlabel('Time')
plt.ylabel('x')

plt.subplot(3, 1, 2)
plt.plot(t, y, label='y(t)')
plt.title('Time Series of y')
plt.xlabel('Time')
plt.ylabel('y')

plt.subplot(3, 1, 3)
plt.plot(t, z, label='z(t)')
plt.title('Time Series of z')
plt.xlabel('Time')
plt.ylabel('z')

plt.tight_layout()
plt.show()


Besides the fact that the spatial coordinates exhibit oscillatory behavior, it is difficult to extract meaningful insights from this representation. Next, we can plot the trajectory and its projection onto different planes

## Trajectory Plots


In [ ]:
from mpl_toolkits.mplot3d import Axes3D

plt.figure(figsize=(12, 8))
plt.subplot(2, 2, 1)
plt.plot(x, y, label='y vs x')
plt.xlabel('x')
plt.ylabel('y')
plt.title('Trajectory - Projection onto the yx plane')

plt.subplot(2, 2, 2)
plt.plot(y, z, label='z vs y')
plt.xlabel('y')
plt.ylabel('z')
plt.title('Trajectory - Projection onto the zy plane')

plt.subplot(2, 2, 3)
plt.plot(x, z, label='z vs x')
plt.xlabel('x')
plt.ylabel('z')
plt.title('Trajectory - Projection onto the zx plane')

plt.tight_layout()
plt.show()


In [ ]:
# Trajectory in 3D
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')
ax.plot(x, y, z, label='Trajectory in 3D Space')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
ax.set_title('3D Trajectory')
plt.show()



Matplotlib has two interfaces:

*   **Axes interface** (object-based, explicit): create a *Figure* and one or more *Axes* objects, then explicitly use methods on these objects to add data, configure limits, set labels etc

*   **pyplot interface** (function-based, implicit): consists of functions in the *pyplot* module. Figure and Axes are manipulated through these functions and are only implicitly present in the background.

## Create an Animated Trajectory Plot


In [8]:
# FuncAnimation is a class in the matplotlib.animation module
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

In [9]:
import matplotlib as mpl


In [10]:
mpl.rcParams['animation.embed_limit'] = 100  # Set embed limit to 100 MB

In [ ]:
# Set up figure and axes. I would like to visualize the animation on the zx plane
fig, ax = plt.subplots()
ax.set_xlim(np.min(x), np.max(x))
ax.set_ylim(np.min(z), np.max(z))
ax.set_xlabel('x')
ax.set_ylabel('z')
ax.set_title('Trajectory - Projection onto the zx plane')

line, = ax.plot([], [], lw=2)

# Initialize the animation
def init():
    line.set_data([], [])
    return line,

# Update function for animation
def update(frame):
    line.set_data(x[:frame], z[:frame])
    return line,

# Create the animation
ani = FuncAnimation(fig, update, frames=len(t), init_func=init, blit=True, interval=20)
HTML(ani.to_jshtml())  # This will display the animation in the notebook




In [12]:
ani.save('phase_space_animation.gif', writer='Pillow', fps=30)

MovieWriter imagemagick unavailable; using Pillow instead.


Now, let's train some ML models for accomplish simple tasks with the Lorenz system, beggining with a common regression task, like predicting the position in the future. 

Independently of the task at hand (and hence the type of model chosen), there are essential steps to be followed. These steps form a workflow that serves as a structured guide applicable to a wide range of machine learning implementations.

## Workflow Summary:

1. **Data collection**: in this case, numerical data is needed to train the ML model. The  numerical solutions obtained in the previous step can be used to feed the model. This data however needs to be structured in a particular way, depending on the task and model used. 

2. **Feature extraction**: features are the inputs to the ML model. Is important to identify the most relevant features for making predictions (*feature importance*) for example, using techniques like SHAP values. On a later stage, more sophisticated features can be added using domain-specific knowledge (*feature engineering*). Basic features include in time forecasting are the raw time series and/or their derivatives. Lagged features can help to capture more complex time dependencies. Advance features include statistical ones like mean, variance or the Fourier transforms. 

3. **Model Selection and Training**: model selection depends on the task at hand.
    * *split the data* : data is divided into training and testing sets

    * *training process* : train the model on the training data. Techniques like **cross-validation** can be used to ensure that the model's performance is not due to overfitting or specific to the training data split.

    * *hyperparameter tuning* : model parameter optimization can be done using grid search, random search, or Bayesian optimization.

4. **Evaluate model performance**: performance of the ML model can be assessed by comparing their predictions with the actual behavior of the system. This can be done using quantitative metrics and visual tools.

    * *Quantitative metrics* : for regression problems popular metrics are MSE, MAE and RMSE. For classification problems metrics include Accuracy, Precision, Recall, F1-Score, and ROC AUC.

    * *Visualization tools* : Prediction vs. Actual - Plot the predicted time series against the actual time series to visually inspect the model’s performance - Residual Analysis and Confusion Matrix (for classification)

5. **Model Refinement** : based on performance insights, model can be tuned by ways of Feature Engineering, combining multiple methods (Ensemble Methods) and Hyperparameter Optimization.

## Predicting Future States (Regression Task)

This is a simple regression task, for that reason I'm using only the time series x(t), y(t) and z(t) as features.

For any ML application, the data needs to be structured in a certain way (depending on the application). For a simple regression task (supervised learning), each sample is represented by features and a target label

In [15]:
# Import necessary libraries
import pandas as pd

# Building the dataframe: include the time variable and the time series for the spatial coordinates as features
df = pd.DataFrame(sol, columns=['x', 'y', 'z'])

# Include the time variable as well
df['t'] = t
df = df[['t','x', 'y', 'z']]

# Create target columns for future states
df['x_future'] = df['x'].shift(-1)
df['y_future'] = df['y'].shift(-1)
df['z_future'] = df['z'].shift(-1)

# Drop the last row since it has NaN targets
df = df.dropna()

In [16]:
df.head()


,t,x,y,z,x_future,y_future,z_future
0,0.00,1.000000,0.000000,0.000000,0.917924,0.266340,0.001266
1,0.01,0.917924,0.266340,0.001266,0.867919,0.511740,0.004670
2,0.02,0.867919,0.511740,0.004670,0.845360,0.744654,0.009883
3,0.03,0.845360,0.744654,0.009883,0.846806,0.972331,0.016842
4,0.04,0.846806,0.972331,0.016842,0.869786,1.201129,0.025688


Now that I have structured the data and identified the features, I have to do the model selection and training part.

Because I would like to perform a simple regression task (predict future values of the time series) I'll start with a simple linear regression model.

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [18]:
# Name the variables properly

# x, y and z are the features.
X = df[['x','y','z']]

# x_future	y_future	z_future are the target labels
Y = df[['x_future','y_future','z_future']]

# Split the data. In this case, the split will be 80-20
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)   # random state for reproducibility

## Feature scaling: each feature in the dataset is standardized by removing the mean and scaling to unit variance. Feature scaling is an important step in data preprocessing, especially for algorithms that are sensitive to the scale of the data

# create an instance of the 'StandardScaler' class
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Scaling the data is important because some algorithms work better or converge faster when the features are on a similar scale.

StandardScaler object calculates the mean and SD of each feature, and scales each data point (e.g if x is an observation of feature x, x_scaled = (x - mu)/sigma) so as the scaled distribution has mean = 0 and sd = 1

the method fit_transform(X_train) fits the scaler by calculating the mean and sd of the X_train set, and subsequently scales the data distribution by applying the corresponding transformation

the method transform(X_test) scales the X_test set using the already fitted scaler. Is important to clarify that the transformation is done by substracting and dividing by the mean and sd of the X_train distribution, not by calculating the mean and sd of X_train. By using the same scaling parameters for both data distributions, we ensure that the model sees data on the same scale during training and testing.

Is important to remember always compute mean and standard deviation from the training data and apply the same transformation to any new data (e.g., validation or test sets). It is possible to reverse the standardization process using scaler.inverse_transform(), which can be useful for interpreting model outputs in the original scale. Standardization is sensitive to outliers. If your data has significant outliers, consider using scaling methods that are robust to outliers, such as RobustScaler

In [19]:
# Because I wasn't sure how to work with de np.array, I converted everything back to pd.DataFrame

X_train = pd.DataFrame(X_train, columns=['x','y','z'])
X_test = pd.DataFrame(X_test, columns=['x','y','z'])
Y_train = pd.DataFrame(Y_train, columns=['x_future','y_future','z_future'])
Y_test = pd.DataFrame(Y_test, columns=['x_future','y_future','z_future'])

# This is not very robust. Better to work with the np.array!

In [28]:
# Import the necessary modules
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [29]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

degree = 2

# Create a pipeline for polynomial regression
pipeline = Pipeline([
    ('poly_features', PolynomialFeatures(degree=degree)),  # Generate polynomial features
    ('scaler', StandardScaler()),  # Optional scaling for features
    ('model', LinearRegression())  # Linear regression model on polynomial features
    ])

# Fit the model on the training set
pipeline.fit(X_train, Y_train['x_future'])

# Predict on the test set
y_hat_for_x = pipeline.predict(X_test)

rmse_for_x = np.sqrt(mean_squared_error(Y_test['x_future'], y_hat_for_x))

print(f'RMSE from Polynomial Regression (degree {degree}): {rmse_for_x}')


RMSE from Polynomial Regression (degree 2): 0.000758170500250553


In [30]:
# Now for y

# Fit the model on the training set
pipeline.fit(X_train, Y_train['y_future'])

# Predict on the test set
y_hat_for_y = pipeline.predict(X_test)

rmse_for_y = np.sqrt(mean_squared_error(Y_test['y_future'], y_hat_for_y))

print(f'RMSE from Polynomial Regression (degree {degree}): {rmse_for_y}')


RMSE from Polynomial Regression (degree 2): 0.023606595025463337


In [31]:
# Last, for z

# Fit the model on the training set
pipeline.fit(X_train, Y_train['z_future'])

# Predict on the test set
y_hat_for_z = pipeline.predict(X_test)

rmse_for_z = np.sqrt(mean_squared_error(Y_test['z_future'], y_hat_for_z))

print(f'RMSE from Polynomial Regression (degree {degree}): {rmse_for_z}')

RMSE from Polynomial Regression (degree 2): 0.012754605166158112


In [32]:
# I have to take into account only the last 600 values for x and z
x_sliced = x[-600:]
z_sliced = z[-600:]

x_sliced.shape, z_sliced.shape

((600,), (600,))

In [ ]:
# Set up figure and axes
fig, ax = plt.subplots()
ax.set_xlim(np.min(x_sliced), np.max(x_sliced))
ax.set_ylim(np.min(z_sliced), np.max(z_sliced))

# Create line objects for the two lines
line1, = ax.plot([], [], lw=2, color='blue')
line2, = ax.plot([], [], lw=2, color='red')

# Initialize the animation
def init():
    line1.set_data([], [])
    line2.set_data([], [])
    return line1, line2

# Update function for animation
def update(frame):
    # Update the data for both lines
    line1.set_data(x_sliced[:frame], z_sliced[:frame])
    line2.set_data(y_hat_for_x[:frame], y_hat_for_z[:frame])
    return line1, line2

ani = FuncAnimation(fig, update, frames=len(t), init_func=init, blit=True, interval=20)
HTML(ani.to_jshtml())  # This will display the animation in the notebook
